In [ ]:
def plot_img_hist(item_dir, num_img=6):
    all_item_dirs = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dirs][:num_img]
  
  #plt.figure(figsize=(10, 10))
    for idx, img_path in enumerate(item_files):
        fig1 = plt.figure(idx,figsize=(10, 10))
        fig1.add_subplot(2, 2, 1)
        img = mpimg.imread(img_path, )
        # img = valeur du pixel pour chaque pixel de l'image, de dimension shape
        plt.imshow(img, cmap='gray')
        fig1.add_subplot(2, 2, 2)
        plt.hist(img.ravel(),bins=256, fc='k', ec='k')
        # img.ravel met tout sous une shape de 1 (liste unique)
  
    plt.tight_layout()

In [ ]:
plot_img_hist(train_pnm_pth,1)

In [ ]:
# Directory actuel
os.path.abspath(os.getcwd())

In [ ]:
list_dir = ["resize_pneumo/", 'resize_pneumo/chest_xray/', 'resize_pneumo/chest_xray/train',
            'resize_pneumo/chest_xray/train/NORMAL', 'resize_pneumo/chest_xray/train/PNEUMONIA',
            'resize_pneumo/chest_xray/test', 'resize_pneumo/chest_xray/test/NORMAL',
            'resize_pneumo/chest_xray/test/PNEUMONIA','resize_pneumo/chest_xray/val',
            'resize_pneumo/chest_xray/val/NORMAL','resize_pneumo/chest_xray/val/PNEUMONIA']

# On crée les directory si ils n'existent pas
for i in list_dir :
    if not (os.path.exists(i)):
        print("dir "+i+ " crée")
        os.mkdir(i)

dirname_work = 'resize_pneumo'
dir_chest_xray = os.path.join('resize_pneumo', 'chest_xray')

train_path_work = os.path.join(dir_chest_xray, 'train')
train_nrml_pth_work = os.path.join(train_path_work, 'NORMAL')
train_pnm_pth_work = os.path.join(train_path_work, 'PNEUMONIA')

val_path_work = os.path.join(dir_chest_xray, 'val')
val_nrml_pth_work = os.path.join(val_path_work, 'NORMAL')
val_pnm_pth_work = os.path.join(val_path_work, 'PNEUMONIA')

test_path_work = os.path.join(dir_chest_xray, 'test')
test_nrml_pth_work = os.path.join(test_path_work, 'NORMAL')
test_pnm_pth_work = os.path.join(test_path_work, 'PNEUMONIA')

In [ ]:
def image_resizing(path_from, path_to, height=500, width=500):
    """ Chaque image doit avoir la même dimension,
    shape pour être traité par un même modèle"""
    size = height, width
    i=1
    files = os.listdir(path_from)
    for file in files: 
        try:
            file_dir = os.path.join(path_from, file)
            file_dir_save = os.path.join(path_to, file)
            img = Image.open(file_dir)
            img = img.resize(size, Image.ANTIALIAS)
            img = img.convert("RGB")
            img.save(file_dir_save) 
            i=i+1
        except:
            continue

In [ ]:
#image_resizing(train_nrml_pth, train_nrml_pth_work, 300, 300)
#image_resizing(train_pnm_pth, train_pnm_pth_work, 300, 300)

In [ ]:
#image_resizing(test_nrml_pth, test_nrml_pth_work, 300, 300)
#image_resizing(test_pnm_pth, test_pnm_pth_work, 300, 300)

In [ ]:
#image_resizing(val_nrml_pth, val_nrml_pth_work, 300, 300)
#image_resizing(val_pnm_pth, val_pnm_pth_work, 300, 300)

In [ ]:
# focal loss 
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
#model.compile(loss=focal_loss(), optimizer='adam', metrics=['accuracy'])

In [ ]:
labels = ['PNEUMONIA', 'NORMAL']
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_data.reshape(-1, img_size_h, img_size_w)[i], cmap='gray')
    if(binary_predictions[i]==test_labels[i]):
        plt.xlabel(labels[binary_predictions[i]], color='blue')
    else:
        plt.xlabel(labels[binary_predictions[i]], color='red')
plt.show()

In [ ]:
def predict_illness(image_path):
    imge = plt.imread(image_path)
    plt.figure()

    img = image.load_img(image_path, target_size=(img_size_h, img_size_w), grayscale = False)
    x = image.img_to_array(img) 
    x = np.expand_dims(x, axis=0)
    #images = x
    images = np.vstack([x])

    classes = model.predict_classes(images, batch_size=10)
    if classes[0][0] == 0:
        textstr = "Prediction: Healthy"
    else:
        textstr = "Prediction: Pneumonia"

    if ('chest_xray/chest_xray/test/NORMAL' in image_path):
        texttruth = "Truth: Healthy"
    else :
        texttruth = "Truth: Pneumonia"
        
    plt.gcf().text(0.9, 0.4, textstr, fontsize=20)
    plt.gcf().text(0.9, 0.6, texttruth, fontsize=20)
    plt.imshow(img, cmap='gray')

In [ ]:
img_p = image.load_img(img_test_nrml, target_size=(img_size_h, img_size_w), grayscale = True)
img_arr_p = np.array(img_p)
img_arr_p = np.expand_dims(img_arr_p, axis=0)
img_arr_p = np.expand_dims(img_arr_p, axis=3)
images_p = np.vstack([img_arr_p])

In [ ]:
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers[:len(model.layers)]]

In [ ]:
activation_model = Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(images_p)

first_layer_activation = activations[0]
plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis', cmap='gray')

In [ ]:
model.layers[:-1]# Droping The Last Dense Layer

In [ ]:
layer_names = []
for layer in model.layers[:-1]:
    layer_names.append(layer.name) 
images_per_row = 16
zipped_layers = zip(layer_names, activations)
for layer_name, layer_activation in zipped_layers: #this loop     
    if layer_name.startswith('conv'):
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // images_per_row
        display_grid = np.zeros((size * n_cols, images_per_row * size))
        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0,:, :, col * images_per_row + row]
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size,
                             row * size : (row + 1) * size] = channel_image
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1],
                            scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
layer_names = []
for layer in model.layers[:-1]:
    layer_names.append(layer.name) 
images_per_row = 16
for layer_name, layer_activation in zip(layer_names, activations):
    if layer_name.startswith('max'):
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // images_per_row
        display_grid = np.zeros((size * n_cols, images_per_row * size))
        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0,:, :, col * images_per_row + row]
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size,
                             row * size : (row + 1) * size] = channel_image
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1],
                            scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')